In [1]:
import numpy as np
import matplotlib as mpl
import seaborn as sns
sns.set()
%matplotlib inline

import pandas as pd

import json
import glob
import os

In [3]:
base_path = "/amd.home/home/i11/zeitz/experiments/tdcch_vldb20/"
paths = glob.glob(base_path + "customization_approx/*.json")
data = [json.load(open(path)) for path in paths]

In [8]:
customization_running_times = pd.DataFrame.from_records([{ 
    'approx': algo['approx'], 
    'approx_threshold': algo['approx_threshold'], 
    'customization_running_time_s': algo['main']['running_time_ms'] / 1000 } 
    for run in data for algo in run["algo_runs"] if algo['algo'] == "Floating TDCCH Customization"])

In [12]:
customization_running_times.pivot(index='approx', columns='approx_threshold', values='customization_running_time_s')

approx_threshold,100,500,1000,5000,10000
approx,,,,,
0.1,1763.834,1766.910,1762.484,1449.090,NaN
0.5,737.649,688.970,631.635,712.222,949.380
1.0,805.900,641.963,572.237,665.808,906.257
5.0,2421.523,1815.343,1420.645,856.114,980.842
10.0,4975.395,3826.408,2757.972,1127.194,1132.046


In [14]:
events = pd.DataFrame.from_records([
    { 
        'approx': algo['approx'], 
        'approx_threshold': algo['approx_threshold'], 
        **event
    } 
    for run in data 
    for algo in run["algo_runs"] if algo['algo'] == "Floating TDCCH Customization"
    for event in algo["main"]["events"]])

In [21]:
events['ipps_per_active_shortcut'] = events['num_ipps_stored'] / events['num_shortcuts_active']

In [22]:
events.groupby(['approx', 'approx_threshold'])['ipps_per_active_shortcut'].mean().unstack()

approx_threshold,100,500,1000,5000,10000
approx,,,,,
0.1,3498.8,3571.1,3599.8,4157.7,nan
0.5,1240.6,1294.6,1363.0,2336.7,3690.7
1.0,910.4,961.6,1067.9,2067.3,3450.5
5.0,604.9,710.7,804.0,1831.1,3276.7
10.0,596.4,580.6,714.1,1786.0,3223.6


In [23]:
events.groupby(['approx', 'approx_threshold'])['ipps_per_active_shortcut'].max().unstack()

approx_threshold,100,500,1000,5000,10000
approx,,,,,
0.1,5037.6,5046.7,5061.7,5399.5,nan
0.5,1309.4,1375.5,1448.0,2576.6,3871.0
1.0,995.8,1065.9,1135.6,2146.8,3607.3
5.0,813.6,944.9,1043.4,1982.4,3501.0
10.0,866.4,821.3,952.9,1947.0,3472.2


In [24]:
events.groupby(['approx', 'approx_threshold'])['num_ipps_stored'].max().unstack()

approx_threshold,100,500,1000,5000,10000
approx,,,,,
0.1,6213019479.0,6165661460.0,6042329973.0,7225173812.0,nan
0.5,2523216250.0,2471313005.0,2944884362.0,4623023299.0,7549048054.0
1.0,2123751829.0,2037731037.0,2573322148.0,4425218409.0,7547094785.0
5.0,1606602946.0,1783939240.0,1894479516.0,4242862766.0,7501678825.0
10.0,1646089178.0,1640741190.0,1906741182.0,3977275697.0,7134169953.0


In [19]:
pd.set_option('display.float_format', lambda x: '%.1f' % x)